In [ ]:
from pathlib import Path
import datetime

import numpy as np
import pandas as pd

In [ ]:
experiment_id = 'hw0'
group_name = 'D'
world_name = 'mountains'
experiment_base_path = Path('/home/klaus/dev/revolve-simulator/l-system/experiments/')

In [ ]:
experiment_path = experiment_base_path / experiment_id
if not experiment_path.exists():
    raise Exception(f'Experiment {str(experiment_path)} does not exist.')

dataframes = []
dataframes_history = []
for run_path in experiment_path.iterdir():
    if run_path.is_dir():
        run_id = run_path.name
        df = pd.read_csv(run_path / 'measures2.txt', sep=' ')
        df.rename(columns={'genome': 'genome_id'}, inplace=True)
        df = pd.pivot_table(df, values='value', index=['generation', 'genome_id'], 
                            columns=['measures'], aggfunc='last')
        
        df_history = pd.read_csv(run_path / 'history.txt', sep=' ')
        df_history.rename(columns={'idgenome': 'genome_id', 'idparent1': 'parent1_id', 
                                   'idparent2': 'parent2_id'}, inplace=True)
        df_history.columns.rename('measures', inplace=True)
        df_history.set_index(['generation', 'genome_id'], inplace=True)
        df_history.drop(df_history.columns[-1], axis=1, inplace=True)
        for c in ['parent1_id', 'parent2_id']:
            df_history[c].replace('N', np.nan, inplace=True)
        
        df = pd.concat([df], keys=[run_id], names=['run_id'])
        df_history = pd.concat([df_history], keys=[run_id], names=['run_id'])
        dataframes.append(df)
        dataframes_history.append(df_history)
        
df = pd.concat(dataframes)

In [ ]:
dataframes[0].head()

In [ ]:
dataframes_history[0].head()

In [ ]:
def get_file_name(group_name: str, world_name: str, experiment_id: str, 
                  run_id: str, with_date: bool = False) -> str:
    if with_date:
        date = '_date=' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    else:
        date = ''
    name = f'experiment_grp={group_name}_wld={world_name}_exp={experiment_id}_run={run_id}{date}.hdf5'
    return name


output_file_name = get_file_name(group_name, world_name, experiment_id, run_id)
output_file_path = experiment_path / output_file_name
df.to_hdf(output_file_path, key='descriptors')

In [ ]:
df = pd.read_hdf(output_file_path, key='descriptors')
df.head()